In [15]:
# Завантаження та підготовка даних
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import warnings
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly, plot_cross_validation_metric, add_changepoints_to_plot

warnings.simplefilter("ignore")

path = os.getcwd()
path = os.path.join(path, 'DATA')

file_paths = {}
for file in os.listdir(path):
    file_paths[file.split('.')[0]] = os.path.join(path, file)
for key, path in file_paths.items():
    exec(f"{key}_df = pd.read_csv(path)")

# Обробка даних
df = train_df.copy()
# Вибір магазину
np.random.seed(123)
ndr = np.random.choice(df.store_nbr.unique())

df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.strftime('%A')

df = df[df['store_nbr'] == ndr]
df = df[['date', 'sales', 'day_of_week']]

df = df[df['sales'] != 0]

# Функція для прогнозування
def forecast_sales(df, periods):
    """
    Функція для прогнозування продажів на задану кількість періодів.

    Args:
        df: DataFrame з даними про продажі.
        periods: Кількість періодів для прогнозування.

    Returns:
        DataFrame з прогнозом продажів.
    """

    # Групування даних
    grouped_df = df.groupby('date').agg({'sales': 'sum', 'day_of_week': 'first'}).reset_index()
    grouped_df.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)  # Перейменування стовпців

    # Створення моделі Prophet
    model = Prophet()
    model.fit(grouped_df)

    # Створення майбутніх дат
    future_sales = model.make_future_dataframe(periods=periods)

    # Прогнозування продажів
    forecast = model.predict(future_sales)

    # Adjust column names if needed (e.g., rename 'ds' to 'date')
    forecast.rename(columns={'ds': 'date'}, inplace=True)

    # Select relevant columns for final forecast_df
    forecast = test.merge(forecast[["ds", "yhat"]], on="ds", how="left")
    forecast_df = forecast

    return model, forecast_df



# Прогнозування на 30, 180, 270, 365 днів
periods_list = [30, 180, 270, 365]

for periods in periods_list:
    model, forecast_df = forecast_sales(df.copy(), periods)
forecast_df
    

21:45:15 - cmdstanpy - INFO - Chain [1] start processing
21:45:15 - cmdstanpy - INFO - Chain [1] done processing


NameError: name 'test' is not defined

In [13]:
test_df

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [ ]:
# Візуалізація прогнозу
    #fig = plot_plotly(model, forecast_df, figsize=(1400, 700))
    #fig.update_layout(showlegend=True)
    #plt.show()

    # Оцінка прогнозу
    benchmark_df = forecast_df.merge(test_df.set_index('date'), on='ds', how='left')

    MAE = mean_absolute_error(benchmark_df['sales'], benchmark_df['yhat'])
    print(f'MAE ({periods} days): {MAE:.2f}')

    MSE = mean_squared_error(benchmark_df['sales'], benchmark_df['yhat'])
    print(f'MSE ({periods} days): {MSE:.2f}')

    MSLE = mean_squared_log_error(benchmark_df['sales'], benchmark_df['yhat'])
    print(f'MSLE ({periods} days): {MSLE:.2f}')

    print()
